# Merge data sets

In [1]:
# Import der Umsatz- und der Wetter-Daten
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

weather = "/workspaces/bakery_prediction/0_DataPreparation/wetter.csv"
weather = pd.read_csv(weather)
print(weather)

test = "/workspaces/bakery_prediction/0_DataPreparation/test.csv"
test = pd.read_csv(test)
#print(test)

train = "/workspaces/bakery_prediction/0_DataPreparation/train.csv"
train = pd.read_csv(train)
print(test)

           Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
0     2012-01-01         8.0      9.8250                   14        58.0
1     2012-01-02         7.0      7.4375                   12         NaN
2     2012-01-03         8.0      5.5375                   18        63.0
3     2012-01-04         4.0      5.6875                   19        80.0
4     2012-01-05         6.0      5.3000                   23        80.0
...          ...         ...         ...                  ...         ...
2596  2019-07-28         3.0     23.3500                   14         5.0
2597  2019-07-29         6.0     25.2500                    7        61.0
2598  2019-07-30         7.0     20.7375                    8        61.0
2599  2019-07-31         6.0     20.4500                    7        61.0
2600  2019-08-01         5.0     21.0625                    9        61.0

[2601 rows x 5 columns]
           id       Datum  Warengruppe
0     1808011  2018-08-01            1
1     180

In [2]:
print(min(train['Datum']))
print(max(train['Datum']))
print(train)


2013-07-01
2018-07-31
           id       Datum  Warengruppe      Umsatz
0     1307011  2013-07-01            1  148.828353
1     1307021  2013-07-02            1  159.793757
2     1307031  2013-07-03            1  111.885594
3     1307041  2013-07-04            1  168.864941
4     1307051  2013-07-05            1  171.280754
...       ...         ...          ...         ...
9329  1712216  2017-12-21            6   87.471228
9330  1712226  2017-12-22            6   71.911652
9331  1712236  2017-12-23            6   84.062223
9332  1712246  2017-12-24            6   60.981969
9333  1712276  2017-12-27            6   34.972644

[9334 rows x 4 columns]


In [3]:
print(min(test['Datum']))
print(max(test['Datum']))
print(test)

2018-08-01
2019-07-30
           id       Datum  Warengruppe
0     1808011  2018-08-01            1
1     1808021  2018-08-02            1
2     1808031  2018-08-03            1
3     1808041  2018-08-04            1
4     1808051  2018-08-05            1
...       ...         ...          ...
1825  1812226  2018-12-22            6
1826  1812236  2018-12-23            6
1827  1812246  2018-12-24            6
1828  1812276  2018-12-27            6
1829  1812286  2018-12-28            6

[1830 rows x 3 columns]


In [4]:
# merge weather data with train and test data
train_data = pd.merge(train,weather, how='outer')
test_data = test# pd.merge(test,Wind, how='left')
print(test_data)

           id       Datum  Warengruppe
0     1808011  2018-08-01            1
1     1808021  2018-08-02            1
2     1808031  2018-08-03            1
3     1808041  2018-08-04            1
4     1808051  2018-08-05            1
...       ...         ...          ...
1825  1812226  2018-12-22            6
1826  1812236  2018-12-23            6
1827  1812246  2018-12-24            6
1828  1812276  2018-12-27            6
1829  1812286  2018-12-28            6

[1830 rows x 3 columns]


In [5]:
# define date thresholds
train_begin_date = "2013-07-01"
train_end_date = "2017-07-31"

validation_begin_date ="2017-08-01"
validation_end_date ="2018-07-31"

test_begin_date = "2018-08-01"
test_end_date = "2019-07-31"


In [6]:
# trim test data to relevant dates and drop rows with missing values
test_data = test_data[(test_data["Datum"] >= test_begin_date) & (test_data["Datum"] <= test_end_date)]
#test_data = test_data.dropna(how='any')
#test_data = test
print(test_data.isna().sum())
print(len(test_data))

id             0
Datum          0
Warengruppe    0
dtype: int64
1830


In [7]:
# trim training data to relevant dates and drop rows with missing values
train_data = train_data[(train_data["Datum"] >= train_begin_date) & (train_data["Datum"] <= train_end_date)]
train_data = train_data.dropna(how='any')
print(train_data.isna().sum())
print(train_data)

id                     0
Datum                  0
Warengruppe            0
Umsatz                 0
Bewoelkung             0
Temperatur             0
Windgeschwindigkeit    0
Wettercode             0
dtype: int64
             id       Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
394   1307011.0  2013-07-01          1.0  148.828353         6.0     17.8375   
395   1307012.0  2013-07-01          2.0  535.856285         6.0     17.8375   
396   1307013.0  2013-07-01          3.0  201.198426         6.0     17.8375   
397   1307014.0  2013-07-01          4.0   65.890169         6.0     17.8375   
398   1307015.0  2013-07-01          5.0  317.475875         6.0     17.8375   
...         ...         ...          ...         ...         ...         ...   
7907  1707301.0  2017-07-30          1.0   65.453279         5.0     24.0000   
7908  1707302.0  2017-07-30          2.0  605.805099         5.0     24.0000   
7909  1707303.0  2017-07-30          3.0  322.250156         5.0   

In [8]:
X_train = train_data[["Warengruppe"]]
X_train = sm.add_constant(X_train)
Y_train = train_data["Umsatz"]

In [9]:
# train model
model = sm.OLS(Y_train,X_train).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     17.68
Date:                Sun, 14 Dec 2025   Prob (F-statistic):           2.66e-05
Time:                        15:48:01   Log-Likelihood:                -34472.
No. Observations:                5399   AIC:                         6.895e+04
Df Residuals:                    5397   BIC:                         6.896e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         219.0414      4.484     48.854      

In [10]:
# Make predictions on the test set
#print(test_data)
X_test = test_data[["Warengruppe"]]
print(X_test)
X_test = sm.add_constant(X_test)  # Add constant to test set for predictions

predictions = model.predict(X_test)
print(len(predictions))


      Warengruppe
0               1
1               1
2               1
3               1
4               1
...           ...
1825            6
1826            6
1827            6
1828            6
1829            6

[1830 rows x 1 columns]
1830


In [11]:
pred_df = pd.DataFrame({
    'Umsatz': predictions
}, index=X_test.index)
X_test2 = test_data[["id"]]
# Merge with original DataFrame
df_with_preds = X_test2.join(pred_df)
#print(X_test2)
print(len(df_with_preds))
print(df_with_preds.isna().sum())
# Save to CSV
write_path = "/workspaces/bakery_prediction/1_DatasetCharacteristics/predictions.csv"
df_with_preds.to_csv(write_path, index=False) 

1830
id        0
Umsatz    0
dtype: int64


In [12]:
#Y_test = test_data["Umsatz"]

# Calculate Mean Absolute Percentage Error (MAPE) for error evaluation
#mape = (abs((Y_test - predictions) / Y_test).mean()) * 100
#print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")